# RAG Hyperparameter Tuning
**Hitrate and Mean Reciprocal Rank**

- Minsearch
- Textsearch Elasticsearch
- Vectorsearch

In [1]:
import minsearch
import json
import pandas as pd


from tqdm.auto import tqdm
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from sklearn.model_selection import train_test_split

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/home/codespace/.local/share/virtualenvs/benefits-claims-assitant-w_hr6DTK/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Ingestion

In [2]:
with open('notebooks/document-with-ids.json', 'r') as file:
    documents = json.load(file)

## Index and mapping Minsearch

In [3]:
index = minsearch.Index(
    text_fields=["question", "answer", "category"],
    keyword_fields=["section", "id"]
)

index.fit(documents)

In [4]:
def minsearch(query):
    boost = {
        'question': 3,
        'answer': 0.5,
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=20
    )

    return results

## RAG flow

In [5]:
q = 'What is the Help to Save scheme?'

In [6]:
client = OpenAI()
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"The Help to Save scheme is a government initiative in the United Kingdom designed to encourage low-income workers to save money. Launched in 2018, it allows eligible individuals to save money in a special savings account, with the government providing a bonus on the savings accrued.\n\nHere are the key features of the Help to Save scheme:\n\n1. **Eligibility**: The scheme is available to people on certain qualifying benefits, including Universal Credit, Working Tax Credit, Income Support, or Jobseeker's Allowance.\n\n2. **Savings Amount**: Participants can save between £1 and £50 per month, with monthly contributions being flexible.\n\n3. **Account Duration**: The savings account can be held for a maximum of four years.\n\n4. **Government Bonus**: For every £1 saved, the government offers a 50% bonus, meaning that individuals can receive up to £1,200 in bonuses at the end of the four-year period if they save the maximum amount allowed.\n\n5. **Access to Funds**: Although the account e

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
You are an expert in United Kingdom Benefit Claims and Medical Negligence Claims. Answer the QUESTION based on the CONTEXT from 
the FAQ databases of Benefits database and NHS claims management. 
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"category: {doc['category']}\nquestion: {doc['question']}\nanswer: {doc['answer']}\nsection: {doc['section']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [8]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [9]:
query = "Can I get sick pay if I'm self-isolating?"

def rag(query):
    search_results = minsearch(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [10]:
rag(query)

"Yes, you can get Statutory Sick Pay if you're self-isolating."

# Evaluating Retrieval
- hit_rate
- Mean Reciprocal Rank

## Minsearch without tuning

In [11]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [12]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [13]:
df_ground_truth = pd.read_csv('notebooks/ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [14]:
ground_truth[10]

{'question': 'Is it possible to appeal?',
 'section': 'general claim benefits',
 'document': '8d000ade-6c2b-571c-aa61-5d38eb463cf8'}

In [15]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

evaluate(ground_truth, lambda q: minsearch(q['question']))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:04<00:00, 420.56it/s]


{'hit_rate': 0.9124087591240876, 'mrr': 0.6409134667669506}

## Hyperparameter tuning

In [ ]:
# Splitting dataset into train, validation, and test sets
df_full_train, df_test = train_test_split(ground_truth, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

# Define search space for hyperparameters
space = {
    'question_boost': hp.uniform('question_boost', 0.5, 3),
    'answer_boost': hp.uniform('answer_boost', 0.5, 3),
    'category_boost': hp.uniform('category_boost', 0.5, 3),
    'num_results': hp.quniform('num_results', 5, 20, 1),
}

In [ ]:
def objective(params):
    boost = {
        'question': params['question_boost'],
        'answer': params['answer_boost'],
        'category': params['category_boost']
    }

    def minsearch(query):
        return index.search(
            query=query,
            filter_dict={},  # Remove filter for now
            boost_dict=boost,
            num_results=int(params['num_results'])
        )

    eval_results = evaluate(df_val, lambda q: minsearch(q['question']))
    
    # We aim to maximize hit_rate and mrr
    return {'loss': -eval_results['hit_rate'], 'status': STATUS_OK}

**Best Paramaters**

In [ ]:
trials = Trials()
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# Apply the best parameters to the whole dataset
best_boost = {
    'question': best_params['question_boost'],
    'answer': best_params['answer_boost'],
    'category': best_params['category_boost']
}
num_results = int(best_params['num_results'])

In [45]:
print("Best Parameters: ", best_params)

Best Parameters:  {'Answer_boost': np.float64(2.5537631763706026), 'Category_boost': np.float64(0.7820035743329506), 'Question_boost': np.float64(1.4411346285867748), 'num_results': np.float64(20.0)}


In [44]:
def minsearch_best(query):
    return index.search(
        query=query,
        filter_dict={},
        boost_dict=best_boost,
        num_results=num_results
    )

# Evaluate on full ground truth dataset
final_results = evaluate(ground_truth, lambda q: minsearch_best(q['question']))
final_results

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:05<00:00, 410.65it/s]


{'hit_rate': 0.9542579075425791, 'mrr': 0.7248949822082627}

## Evaluating Text Search

In [19]:
with open('notebooks/document-with-ids.json', 'r') as file:
    documents = json.load(file)

documents[10]

{'category': 'Temporarily unable to work',
 'question': "Can I get sick pay if I'm self-isolating?",
 'answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
 'section': 'general claim benefits',
 'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0'}

## Index and Mapping Elasticsearch

In [24]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "answer": {"type": "text"},
            "category": {"type": "text"},
            "question": {"type": "text"},
            "section": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "benefit-claims"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'benefit-claims'})

In [21]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 425/425 [00:08<00:00, 49.14it/s]


In [22]:
def elastic_search(query, section):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "answer", "category"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "section": section
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [23]:
elastic_search(
    query="Can causation be established if multiple factors contributed to the harm?",
    section="nhs claim benefits"
)

[{'category': 'Causation',
  'question': 'Can causation be established if multiple factors contributed to the harm?',
  'answer': 'Yes causation can be established if the negligent act was a significant factor in causing the harm even if other factors were involved.',
  'section': 'nhs claim benefits',
  'id': 'c3556590-7d02-5bd7-8be4-ca64bb135162'},
 {'category': 'Causation',
  'question': 'Can causation be established if the harm was only partially due to negligence?',
  'answer': 'Yes causation can still be established if negligence was a significant contributing factor to the harm even if not the sole cause.',
  'section': 'nhs claim benefits',
  'id': '03b12538-fe8b-5ba1-8c5e-b77cb74518de'},
 {'category': 'Causation',
  'question': 'Can causation be established if the harm was a known risk?',
  'answer': 'Causation may be harder to prove if the harm was a known risk unless it can be shown that negligence increased the risk.',
  'section': 'nhs claim benefits',
  'id': '045ffa5c-85

In [24]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [25]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [26]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

evaluate(ground_truth, lambda q: elastic_search(q['question'], q['section']))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:07<00:00, 287.16it/s]


{'hit_rate': 0.7732360097323601, 'mrr': 0.616999188969993}

## Hyperparameter tuning elasticsearch

In [ ]:
# Splitting dataset into train, validation, and test sets
df_full_train, df_test = train_test_split(ground_truth, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [ ]:
# Define search space for hyperparameters
space = {
    'question_boost': hp.uniform('question_boost', 1, 5),
    'answer_boost': hp.uniform('answer_boost', 1, 3),
    'category_boost': hp.uniform('category_boost', 0.5, 2),
    'size': hp.quniform('size', 1, 10, 1)
}

In [ ]:
# ElasticSearch function with parameters
def elastic_search(query, section, params):
    search_query = {
        "size": int(params['size']),
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": [
                            f"Question^{params['question_boost']}",
                            f"Answer^{params['answer_boost']}",
                            f"Category^{params['category_boost']}"
                        ],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "section": section
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    result_docs = [hit['_source'] for hit in response['hits']['hits']]
    return result_docs

In [57]:
# Objective function for hyperparameter tuning
def objective(params):
    def search_with_params(query, section):
        return elastic_search(query, section, params)

    eval_results = evaluate(df_val, lambda q: search_with_params(q['question'], q['section']))

    return {'loss': -eval_results['hit_rate'], 'status': STATUS_OK}

# Run hyperopt to find the best hyperparameters
trials = Trials()
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# Apply the best parameters on the entire ground_truth dataset
def elastic_search_best(query, section):
    return elastic_search(query, section, best_params)

# Evaluate on full dataset
final_results = evaluate(ground_truth, lambda q: elastic_search_best(q['question'], q['section']))

  0%|                                                                                                  | 0/50 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/411 [00:00<?, ?it/s]

 11%|#1        | 46/411 [00:00<00:00, 454.43it/s]

 25%|##5       | 103/411 [00:00<00:00, 521.85it/s]

 39%|###8      | 160/411 [00:00<00:00, 541.56it/s]

 53%|#####3    | 218/411 [00:00<00:00, 555.26it/s]

 67%|######7   | 277/411 [00:00<00:00, 564.63it/s]

 82%|########1 | 335/411 [00:00<00:00, 568.95it/s]

 96%|#########5| 394/411 [00:00<00:00, 575.06it/s]

100%|##########| 411/411 [00:00<00:00, 558.22it/s]


  2%|█▍                                                                     | 1/50 [00:00<00:36,  1.34trial/s, best loss: -0.9148418491484185]

  0%|          | 0/411 [00:00<?, ?it/s]

 15%|#4        | 60/411 [00:00<00:00, 590.83it/s]

 29%|##9       | 120/411 [00:00<00:00, 477.21it/s]

 43%|####2     | 175/411 [00:00<00:00, 506.21it/s]

 57%|#####6    | 234/411 [00:00<00:00, 534.65it/s]

 70%|#######   | 289/411 [00:00<00:00, 516.91it/s]

 83%|########3 | 342/411 [00:00<00:00, 499.90it/s]

 98%|#########8| 403/411 [00:00<00:00, 531.52it/s]

100%|##########| 411/411 [00:00<00:00, 521.41it/s]


  4%|██▊                                                                    | 2/50 [00:01<00:37,  1.29trial/s, best loss: -0.9148418491484185]

...

  0%|          | 0/411 [00:00<?, ?it/s]

 15%|#4        | 61/411 [00:00<00:00, 602.57it/s]

 30%|##9       | 122/411 [00:00<00:00, 596.94it/s]

 44%|####4     | 182/411 [00:00<00:00, 588.01it/s]

 59%|#####8    | 241/411 [00:00<00:00, 584.05it/s]

 73%|#######3  | 301/411 [00:00<00:00, 586.90it/s]

 88%|########7 | 361/411 [00:00<00:00, 591.14it/s]

100%|##########| 411/411 [00:00<00:00, 552.52it/s]


 98%|████████████████████████████████████████████████████████████████████▌ | 49/50 [00:36<00:00,  1.35trial/s, best loss: -0.9245742092457421]

  0%|          | 0/411 [00:00<?, ?it/s]

 12%|#2        | 51/411 [00:00<00:00, 502.34it/s]

 25%|##4       | 102/411 [00:00<00:00, 505.86it/s]

 38%|###7      | 155/411 [00:00<00:00, 513.58it/s]

 50%|#####     | 207/411 [00:00<00:00, 484.94it/s]

 62%|######2   | 256/411 [00:00<00:00, 452.27it/s]

 75%|#######5  | 309/411 [00:00<00:00, 474.91it/s]

 90%|########9 | 369/411 [00:00<00:00, 513.03it/s]

100%|##########| 411/411 [00:00<00:00, 504.20it/s]


100%|██████████████████████████████████████████████████████████████████████| 50/50 [00:37<00:00,  1.33trial/s, best loss: -0.9245742092457421]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:03<00:00, 540.30it/s]


In [60]:
print("Best Parameters: ", best_params)
print("Final Results: ", final_results)

Best Parameters:  {'Answer_boost': np.float64(2.536150531766629), 'Category_boost': np.float64(1.561542883343601), 'Question_boost': np.float64(3.287329862636004), 'size': np.float64(10.0)}
Final Results:  {'hit_rate': 0.9192214111922141, 'mrr': 0.7220206233344916}


## Vector Search without tuning

In [19]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/home/codespace/.local/share/virtualenvs/benefits-claims-assitant-w_hr6DTK/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
v = model.encode("Can I get sick pay if I'm self-isolating?")

In [21]:
len(v)

384

In [25]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "answer": {"type": "text"},
            "category": {"type": "text"},
            "question": {"type": "text"},
            "section": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "benefit-claims"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'benefit-claims'})

In [26]:
documents[0]

{'category': 'Manage existing benefit',
 'question': 'How do I update my benefit information?',
 'answer': 'You can update your benefit information online through your account.',
 'section': 'general claim benefits',
 'id': '30eada08-5708-5c5c-9df8-0f7d5d4dc131'}

In [27]:
for doc in tqdm(documents):
    question = doc['question']
    answer = doc['answer']
    qt = question + ' ' + answer

    doc['question_vector'] = model.encode(question)
    doc['answer_vector'] = model.encode(answer)
    doc['question_answer_vector'] = model.encode(qt)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 425/425 [00:20<00:00, 20.87it/s]


In [28]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 425/425 [00:11<00:00, 37.45it/s]


In [29]:
query = "Can I get sick pay if I'm self-isolating?"
v_q = model.encode(query)

In [30]:
def elastic_search_knn(field, vector, section):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "Section": section
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["answer", "section", "question", "category", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [31]:
elastic_search_knn('question_vector', v_q, 'general claim benefits')

[]

In [32]:
ground_truth[10]

{'question': 'Is it possible to appeal?',
 'section': 'general claim benefits',
 'document': '8d000ade-6c2b-571c-aa61-5d38eb463cf8'}

In [33]:
def question_vector_knn(q):
    question = q['question']
    section = q['section']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, section)

In [34]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [35]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [36]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [37]:
evaluate(ground_truth, question_vector_knn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:36<00:00, 56.97it/s]


{'hit_rate': 0.0, 'mrr': 0.0}

In [38]:
def answer_vector_knn(q):
    question = q['question']
    section = q['section']

    v_q = model.encode(question)

    return elastic_search_knn('answer_vector', v_q, section)
evaluate(ground_truth, answer_vector_knn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:33<00:00, 60.45it/s]


{'hit_rate': 0.0, 'mrr': 0.0}

In [39]:
def question_answer_vector_knn(q):
    question = q['question']
    section = q['section']

    v_q = model.encode(question)

    return elastic_search_knn('question_answer_vector', v_q, section)

evaluate(ground_truth, question_answer_vector_knn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:33<00:00, 60.90it/s]


{'hit_rate': 0.0, 'mrr': 0.0}

In [40]:
def elastic_search_knn_combined(vector, section):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "section": section
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'answer_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_answer_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "section": section
                    }
                }
            }
        },
        "_source": ["answer", "section", "question", "category", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [41]:
def vector_combined_knn(q):
    question = q['question']
    section = q['section']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, section)

evaluate(ground_truth, vector_combined_knn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:44<00:00, 46.38it/s]


{'hit_rate': 0.9211678832116789, 'mrr': 0.8021573398215729}

question_answer_vector : {'hit_rate': 0.9304136253041363, 'mrr': 0.8123276561232763}

### Hyperparameter tuning for question and answer vector

In [88]:
# Splitting dataset into train, validation, and test sets
df_full_train, df_test = train_test_split(ground_truth, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [103]:
space = {
    'k': hp.quniform('k', 1, 50, 1),  
    'num_candidates': hp.quniform('num_candidates', 1000, 10000, 1000)  
}

In [104]:
def elastic_search_knn(field, vector, section, k, num_candidates):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": int(k),  
        "num_candidates": int(num_candidates), 
        "filter": {
            "term": {
                "Section": section
            }
        }
    }
    
    search_query = {
        "knn": knn,
        "_source": ["answer", "section", "question", "category", "id"]
    }
    
    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [105]:
def question_answer_vector_knn(q, k, num_candidates):
    question = q['question']
    section = q['section']
    v_q = model.encode(question)  # Vector representation of the question
    
    return elastic_search_knn('question_answer_vector', v_q, section, k, num_candidates)

def evaluate(ground_truth, search_function, k, num_candidates):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q, k, num_candidates)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


In [106]:
def objective(params):
    k = params['k']
    num_candidates = params['num_candidates']
    
    metrics = evaluate(df_val, question_answer_vector_knn, k, num_candidates)
    
    return {
        'loss': -metrics['hit_rate'],  # We minimize the negative hit rate
        'status': STATUS_OK
    }


In [107]:
trials = Trials()

# Optimize the hyperparameters
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,  # Number of evaluations
    trials=trials
)

print("Best parameters:", best)

  0%|                                                                                                  | 0/50 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/411 [00:00<?, ?it/s]

  1%|          | 4/411 [00:00<00:10, 39.19it/s]

  2%|2         | 9/411 [00:00<00:09, 43.17it/s]

  3%|3         | 14/411 [00:00<00:09, 43.23it/s]

  5%|4         | 19/411 [00:00<00:10, 39.10it/s]

  6%|5         | 24/411 [00:00<00:09, 41.78it/s]

  7%|7         | 29/411 [00:00<00:09, 39.96it/s]

  8%|8         | 34/411 [00:00<00:09, 41.63it/s]

  9%|9         | 39/411 [00:00<00:08, 42.25it/s]

 11%|#         | 44/411 [00:01<00:08, 43.24it/s]

 12%|#1        | 49/411 [00:01<00:08, 44.30it/s]

 13%|#3        | 54/411 [00:01<00:07, 44.94it/s]

 14%|#4        | 59/411 [00:01<00:08, 42.07it/s]

 16%|#5        | 64/411 [00:01<00:09, 36.71it/s]

 17%|#6        | 69/411 [00:01<00:08, 38.75it/s]

 18%|#8        | 74/411 [00:01<00:08, 40.53it/s]

 19%|#9        | 79/411 [00:01<00:07, 42.25it/s]

 20%|##        | 84/411 [00:02<00:07, 43.68it/s]

 22%|##1       | 89/411 [00:02<00:07, 43.85it/s]

 23%|##2       | 94/411 [00:02<00:07, 44.26it/s]

 24%|##4   

  2%|█▍                                                                     | 1/50 [00:09<07:42,  9.43s/trial, best loss: -0.9635036496350365]

  0%|          | 0/411 [00:00<?, ?it/s]

  2%|1         | 7/411 [00:00<00:06, 61.21it/s]

  3%|3         | 14/411 [00:00<00:06, 61.26it/s]

  5%|5         | 21/411 [00:00<00:06, 61.70it/s]

  7%|6         | 28/411 [00:00<00:06, 61.83it/s]

  9%|8         | 35/411 [00:00<00:06, 61.23it/s]

 10%|#         | 42/411 [00:00<00:06, 60.43it/s]

 12%|#1        | 49/411 [00:00<00:06, 59.93it/s]

 14%|#3        | 56/411 [00:00<00:05, 60.37it/s]

 15%|#5        | 63/411 [00:01<00:05, 60.96it/s]

 17%|#7        | 70/411 [00:01<00:06, 54.75it/s]

 18%|#8        | 76/411 [00:01<00:06, 54.98it/s]

 20%|#9        | 82/411 [00:01<00:05, 55.36it/s]

 22%|##1       | 89/411 [00:01<00:05, 56.95it/s]

 23%|##3       | 95/411 [00:01<00:05, 54.04it/s]

 25%|##4       | 102/411 [00:01<00:05, 56.30it/s]

 27%|##6       | 109/411 [00:01<00:05, 57.74it/s]

 28%|##7       | 115/411 [00:01<00:05, 58.02it/s]

 29%|##9       | 121/411 [00:02<00:05, 57.46it/s]

 31%|###       | 127/411 [00:02<00:05, 55.30it/s]

 32%|

  4%|██▊                                                                    | 2/50 [00:16<06:28,  8.10s/trial, best loss: -0.9635036496350365]

...

 96%|███████████████████████████████████████████████████████████████████▏  | 48/50 [07:08<00:17,  8.97s/trial, best loss: -0.9635036496350365]

  0%|          | 0/411 [00:00<?, ?it/s]

  1%|1         | 5/411 [00:00<00:08, 47.50it/s]

  2%|2         | 10/411 [00:00<00:09, 41.76it/s]

  4%|3         | 15/411 [00:00<00:09, 43.52it/s]

  5%|4         | 20/411 [00:00<00:08, 44.96it/s]

  6%|6         | 25/411 [00:00<00:08, 46.28it/s]

  7%|7         | 30/411 [00:00<00:08, 45.70it/s]

  9%|8         | 35/411 [00:00<00:08, 46.03it/s]

 10%|9         | 40/411 [00:00<00:07, 46.70it/s]

 11%|#         | 45/411 [00:00<00:07, 46.15it/s]

 12%|#2        | 50/411 [00:01<00:08, 44.10it/s]

 13%|#3        | 55/411 [00:01<00:08, 43.51it/s]

 15%|#4        | 60/411 [00:01<00:07, 44.42it/s]

 16%|#5        | 65/411 [00:01<00:08, 42.62it/s]

 17%|#7        | 70/411 [00:01<00:07, 43.55it/s]

 18%|#8        | 75/411 [00:01<00:07, 44.05it/s]

 19%|#9        | 80/411 [00:01<00:07, 44.67it/s]

 21%|##        | 85/411 [00:01<00:07, 45.49it/s]

 22%|##1       | 90/411 [00:02<00:07, 45.65it/s]

 23%|##3       | 95/411 [00:02<00:06, 45.80it/s]

 24%|##4  

 98%|████████████████████████████████████████████████████████████████████▌ | 49/50 [07:18<00:09,  9.03s/trial, best loss: -0.9635036496350365]

  0%|          | 0/411 [00:00<?, ?it/s]

  1%|1         | 5/411 [00:00<00:08, 47.80it/s]

  2%|2         | 10/411 [00:00<00:08, 47.83it/s]

  4%|3         | 15/411 [00:00<00:08, 48.06it/s]

  5%|4         | 20/411 [00:00<00:08, 48.04it/s]

  6%|6         | 25/411 [00:00<00:07, 48.53it/s]

  7%|7         | 30/411 [00:00<00:08, 46.79it/s]

  9%|8         | 35/411 [00:00<00:08, 42.46it/s]

 10%|9         | 40/411 [00:00<00:08, 44.40it/s]

 11%|#         | 45/411 [00:00<00:07, 45.92it/s]

 12%|#2        | 50/411 [00:01<00:07, 46.67it/s]

 13%|#3        | 55/411 [00:01<00:07, 47.00it/s]

 15%|#4        | 60/411 [00:01<00:07, 47.56it/s]

 16%|#5        | 65/411 [00:01<00:07, 47.90it/s]

 17%|#7        | 70/411 [00:01<00:07, 44.49it/s]

 18%|#8        | 75/411 [00:01<00:07, 44.86it/s]

 19%|#9        | 80/411 [00:01<00:07, 45.67it/s]

 21%|##        | 85/411 [00:01<00:07, 46.37it/s]

 22%|##1       | 90/411 [00:01<00:06, 46.53it/s]

 23%|##3       | 95/411 [00:02<00:06, 46.65it/s]

 24%|##4  

100%|██████████████████████████████████████████████████████████████████████| 50/50 [07:27<00:00,  8.94s/trial, best loss: -0.9635036496350365]
Best parameters: {'k': np.float64(23.0), 'num_candidates': np.float64(9000.0)}


In [108]:
best

{'k': np.float64(23.0), 'num_candidates': np.float64(9000.0)}

In [109]:
best_k = int(best['k'])
best_num_candidates = int(best['num_candidates'])

# Evaluate on test set with best hyperparameters
test_metrics = evaluate(df_test, question_answer_vector_knn, best_k, best_num_candidates)
print("Test metrics:", test_metrics)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 411/411 [00:09<00:00, 45.45it/s]

Test metrics: {'hit_rate': 0.9562043795620438, 'mrr': 0.8328158189472059}


In [110]:
final_metrics = evaluate(ground_truth, question_answer_vector_knn, best_k, best_num_candidates)
print("final metrics:", final_metrics)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 2055/2055 [00:45<00:00, 45.21it/s]

final metrics: {'hit_rate': 0.9610705596107056, 'mrr': 0.8165805430038999}


SELECTED RETRIEVAL: QUESTION ANSWER VECTOR OF VECTOR ELASTICSEARCH: 
**`{'hit_rate': 0.9610705596107056, 'mrr': 0.8165805430038999}`**